# CONTROL EJECUCIÓN OBRAS SIIF VS ICARO

Conjunto de reportes que, mediante el cruce de información entre SIIF e Icaro, permiten un control ex-post de la ejecución presupuestaria de obras (partidas 421 y 422). Los controles se efectúan a nivel estructura programática, carga de comprobantes de gastos y carga de PA6 (incluyendo su regularización)

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [1]:
from invicoctrlpy.icaro.icaro_vs_siif.icaro_vs_siif import IcaroVsSIIF

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

from pivottablejs import pivot_ui

EJERCICIO = '2022'
UPDATE_DB = False
icaro_vs_siif = IcaroVsSIIF(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Controles

### Ejecución Anual por Estructura

Control de la ejecución anual a nivel estructura programática (incluyendo fuente financiera). La información del SIIF es obtenida del reporte rf602. Las diferencias que pudieran surgir tendrán distintos orígenes, como ser:
 - falta de carga de comprobantes en Icaro (diferencia positiva)
 - carga incorrecta tanto en SIIF como en Icaro

El control que sigue a continuación permitirá disminuir alguna (o todas) las diferencias detectadas en el presente. En caso de que las mismas prevalezcan, deberá cruzar la información de las estructuras afectadas. Para ello, se recomienda utilizar los informes (por ej. informe por partidas) y reportes (por ej. ...) del SIIF, por un lado, y el archivo de tabla dinámica de Icaro que acompaña al presente reporte, por otro lado.

In [2]:
ejec_anual = icaro_vs_siif.control_ejecucion_anual()
itables.show(ejec_anual, column_filters="footer", dom="lrtip")

### Comprobantes gastos SIIF vs Icaro

Control en la carga de comprobantes de gastos. La información del SIIF es obtenida de los reportes rcg01_uejp y gto_rpa03g (gpo 400). El presente reporte permite detectar 8 tipos de errores (nro_comprobante, fecha, importe, cta_cte, fuente, cuit, tipo_comprobante, partida) cuyo orígenes podrán ser:
 - falta o error en la carga de Icaro
 - error en la carga de SIIF
 - utilización de MAPs en SIIF. Es importante tener en cuenta que Icaro no registra este tipo de comprobantes lo cual genera diferencias tanto en la propia carga del MAP como en los comprobantes sobre los cuales recae.

In [3]:
comprobantes = icaro_vs_siif.control_comprobantes()
itables.show(comprobantes, column_filters="footer", dom="lrtip")

In [4]:
test = icaro_vs_siif.import_siif_comprobantes()
itables.show(test, column_filters="footer", dom="lrtip")

### Control PA6

Control tanto en los PA6 como en su posterior regularización. La información del SIIF es obtenida de los reportes rcg01_uejp y rfondo07tp. El presente reporte permite detectar 9 tipos de errores (nro_fondo, fecha_pa6, fecha_reg, importe_pa6, importe_reg, cta_cte, fuente, cuit, tipo_comprobante) cuyo principal orígen provendrá de una incorrecta carga en Icaro.

In [5]:
control_pa6 = icaro_vs_siif.control_pa6()
itables.show(control_pa6, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [6]:
with pd.ExcelWriter('icaro_vs_siif.xlsx') as writer:
    ejec_anual.to_excel(writer, sheet_name='ejec_anual_estructura', index=False)
    comprobantes.to_excel(writer, sheet_name='comprobantes_gtos', index=False)
    control_pa6.to_excel(writer, sheet_name='control_pa6', index=False)

### Tablas Dinámicas

In [7]:
# icaro_sin_pa6 = icaro_vs_siif.icaro.copy()
# icaro_sin_pa6 = icaro_sin_pa6.loc[icaro_sin_pa6['tipo'] != 'PA6']
# pivot_ui(icaro_sin_pa6, 'tabla_dinamica_icaro_sin_pa6.html')